In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import average_precision_score
import warnings
warnings.filterwarnings('ignore')

1. Import data
2. Select Test and train data
3. Divide data in target and features
4. Search and Select Relevant Features // Not focused
5. Develop Ranks

--> Create ranks using ranking rules

9. Develope ranking method (ART Method)
10. Derive absoluate ranks to check accuracy
11. Predict ranks of test set and find accuracy (Do not calculate but predict)


Predict Rank -> Check Relevancy -> Use time to remove ties

In [2]:
data1900 = pd.read_csv("F:\\ML\\MultiTarget\\data1900.csv")
Test  = data1900[data1900.Category=='Test']
Train = data1900[data1900.Category == 'Train']
print("Test Shape:", Test.shape,"\t train shape:", Train.shape)

('Test Shape:', (465, 157), '\t train shape:', (1500, 157))


In [3]:
# Features starts from 33rd columns
Test_features  = Test.iloc[:,36:]
Train_features = Train.iloc[:,36:]

# Classification targets (Reasoner to be predicted)
Test_Targets_c = Test[filter(lambda x: (x.endswith('CORR')) , Test.columns.tolist())]
Train_Targets_c = Train[filter(lambda x: (x.endswith('CORR')) , Train.columns.tolist())]

for t in Test_Targets_c.columns.tolist():
    Test_Targets_c.loc[Test_Targets_c[t] != 'Correct', t] = 0
    Test_Targets_c.loc[Test_Targets_c[t] == 'Correct', t] = 1
    
    Train_Targets_c.loc[Train_Targets_c[t] != 'Correct', t] = 0
    Train_Targets_c.loc[Train_Targets_c[t] == 'Correct', t] = 1

lbl = preprocessing.LabelEncoder()

for t in Test_features.columns.tolist():
    if(Train_features[t].dtype == 'O'):
        Train_features[t] = lbl.fit_transform(Train_features[t])
        Test_features[t] = lbl.fit_transform(Test_features[t])
'''
for t in Test_Targets_c.columns.tolist():
    if(Train_Targets_c[t].dtype == 'object'):
        Test_Targets_c[t] = lbl.fit_transform(Test_Targets_c[t])
        Train_Targets_c[t] = lbl.fit_transform(Train_Targets_c[t])
'''    

#Regressor targets (Execution time of each reasoner)
Test_Targets_r = Test[filter(lambda x: (x.endswith('ETime')) , Test.columns.tolist())]
Train_Targets_r = Train[filter(lambda x: (x.endswith('ETime')) , Train.columns.tolist())]


## Calculation of reasoner ranks using following rules
1. Arrange reasoners according to relevancy
    Correct > Unexpected > Timeout >Execution Error
2. Arrange each reasoner according to execution time

In [4]:
# Select data

Train_Targets_c2 = Train[filter(lambda x: (x.endswith('CORR')) , Train.columns.tolist())]
Test_Targets_c2 = Test[filter(lambda x: (x.endswith('CORR')) , Test.columns.tolist())]

#print(Train_Targets_c2.tail())

for t in Test_Targets_c.columns.tolist():
    Test_Targets_c2.loc[Test_Targets_c2[t] == 'Correct', t] = 1
    Test_Targets_c2.loc[Test_Targets_c2[t] == 'Unexpected', t] = 2
    Test_Targets_c2.loc[Test_Targets_c2[t] == 'Timeout', t] = 3
    Test_Targets_c2.loc[Test_Targets_c2[t] == 'Exec_err', t] = 4
    
    Train_Targets_c2.loc[Train_Targets_c2[t] == 'Correct', t] = 1
    Train_Targets_c2.loc[Train_Targets_c2[t] == 'Unexpected', t] = 2
    Train_Targets_c2.loc[Train_Targets_c2[t] == 'Timeout', t] = 3
    Train_Targets_c2.loc[Train_Targets_c2[t] == 'Exec_err', t] = 4
    
Train_Targets_c2.tail()

,Racer_CORR,ELepHant_CORR,TrOWL_CORR,JFact_CORR,MOReHermiT_CORR,Konclude_CORR,FaCT++_CORR,ELK_CORR,Pellet-OA4_CORR,HermiT-OA4_CORR
1959,1,4,2,2,1,1,1,2,1,1
1960,1,4,2,3,1,1,1,2,3,1
1961,1,4,2,3,1,1,1,2,3,3
1963,1,4,2,2,1,1,1,2,1,1
1964,1,4,1,3,1,1,1,1,1,1


In [5]:
reasoner_ranks_train = Train_Targets_c2.copy(deep=True)

for index, row in Train_Targets_c2.iterrows():
    one_df   = pd.DataFrame(columns = ['r_name','r_time','r_rank'])
    two_df   = pd.DataFrame(columns = ['r_name','r_time','r_rank'])
    three_df = pd.DataFrame(columns = ['r_name','r_time','r_rank'])
    four_df  = pd.DataFrame(columns = ['r_name','r_time','r_rank'])

    
    for c in Train_Targets_c2.columns:
        #print(Test_Targets_c2.loc[index][c])
        r = c.split('_')[0]
        #print(Test_Targets_c2.loc[index])
 #       print(c,Test_Targets_c2.loc[index][c])
        if(Train_Targets_c2.loc[index][c] == 1):
            #one_l.append(c.split('_')[0])
            one_df = one_df.append({'r_name': r,'r_time':Train_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
        if(Train_Targets_c2.loc[index][c] == 2):
            #two_l.append(c.split('_')[0])
            two_df = two_df.append({'r_name': r,'r_time':Train_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
        if(Train_Targets_c2.loc[index][c] == 3):
            #three_l.append(c.split('_')[0])
            three_df = three_df.append({'r_name': r,'r_time':Train_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
        if(Train_Targets_c2.loc[index][c] == 4):
            #four_l.append(c.split('_')[0])
            four_df = four_df.append({'r_name': r,'r_time':Train_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
   
    one_df.sort_values(['r_time'], inplace=True)
    two_df.sort_values(['r_time'], inplace=True)
    three_df.sort_values(['r_time'], inplace=True)
    four_df.sort_values(['r_time'], inplace=True)
    #print(one_df.head()) 
    r=1
    for i, r_t in one_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_train.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    for i, r_t in two_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_train.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    for i, r_t in three_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_train.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    for i, r_t in four_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_train.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    

In [6]:
reasoner_ranks_test = Test_Targets_c2.copy(deep=True)

for index, row in Test_Targets_c2.iterrows():
    one_df   = pd.DataFrame(columns = ['r_name','r_time','r_rank'])
    two_df   = pd.DataFrame(columns = ['r_name','r_time','r_rank'])
    three_df = pd.DataFrame(columns = ['r_name','r_time','r_rank'])
    four_df  = pd.DataFrame(columns = ['r_name','r_time','r_rank'])

    
    for c in Test_Targets_c2.columns:
        #print(Test_Targets_c2.loc[index][c])
        r = c.split('_')[0]
        #print(Test_Targets_c2.loc[index])
 #       print(c,Test_Targets_c2.loc[index][c])
        if(Test_Targets_c2.loc[index][c] == 1):
            #one_l.append(c.split('_')[0])
            one_df = one_df.append({'r_name': r,'r_time':Test_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
        if(Test_Targets_c2.loc[index][c] == 2):
            #two_l.append(c.split('_')[0])
            two_df = two_df.append({'r_name': r,'r_time':Test_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
        if(Test_Targets_c2.loc[index][c] == 3):
            #three_l.append(c.split('_')[0])
            three_df = three_df.append({'r_name': r,'r_time':Test_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
        if(Test_Targets_c2.loc[index][c] == 4):
            #four_l.append(c.split('_')[0])
            four_df = four_df.append({'r_name': r,'r_time':Test_Targets_r.loc[index][r + "_ETime"]}, ignore_index=True)
   
    one_df.sort_values(['r_time'], inplace=True)
    two_df.sort_values(['r_time'], inplace=True)
    three_df.sort_values(['r_time'], inplace=True)
    four_df.sort_values(['r_time'], inplace=True)
    #print(one_df.head()) 
    r=1
    for i, r_t in one_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_test.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    for i, r_t in two_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_test.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    for i, r_t in three_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_test.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    for i, r_t in four_df.iterrows():
        #print(i+1,r_t['r_name'])
        reasoner_ranks_test.loc[index][r_t['r_name']+'_CORR'] = r
        r = r+1
    

In [7]:
reasoner_ranks_train.tail()

,Racer_CORR,ELepHant_CORR,TrOWL_CORR,JFact_CORR,MOReHermiT_CORR,Konclude_CORR,FaCT++_CORR,ELK_CORR,Pellet-OA4_CORR,HermiT-OA4_CORR
1959,5,10,8,9,1,2,3,7,6,4
1960,5,10,7,8,1,2,3,6,9,4
1961,4,10,6,8,1,2,3,5,9,7
1963,5,10,8,9,2,1,3,7,6,4
1964,8,10,4,9,2,3,6,1,5,7


In [42]:
clf_rf = RandomForestClassifier(n_estimators=100,random_state=45)
mclf_rf = MultiOutputClassifier(clf_rf, n_jobs=-1)
mclf_rf.fit(Train_features, reasoner_ranks_train)

print(mclf_rf.score(Test_features, reasoner_ranks_test))

0.296774193548


In [43]:
predicted_ranks_for_map = pd.DataFrame(data = mclf_rf.predict(Test_features), columns = reasoner_ranks_test.columns)

In [40]:
predicted_ranks_for_map.head(10)

,Racer_CORR,ELepHant_CORR,TrOWL_CORR,JFact_CORR,MOReHermiT_CORR,Konclude_CORR,FaCT++_CORR,ELK_CORR,Pellet-OA4_CORR,HermiT-OA4_CORR
0,6.96,9.000000,1.980000,9.7300,4.370000,1.230,7.710000,3.010,5.440000,5.490000
1,4.34,8.657583,3.210000,5.1875,7.102583,2.410,9.037583,1.100,6.070000,7.965250
2,3.15,1.110000,5.180000,7.8900,9.780000,1.690,5.520000,4.195,8.690000,7.040000
3,8.99,8.349833,5.202667,6.2900,5.680000,1.935,4.900000,1.480,7.916667,5.186667
4,7.03,8.438167,8.973333,6.5070,3.260000,2.480,6.171667,1.090,6.404333,5.245000
5,5.13,9.990000,2.100000,8.1600,6.496000,1.480,3.440000,5.570,8.080000,4.646667
6,6.86,9.030000,3.050000,9.9700,4.065000,2.080,7.790000,1.210,5.610000,5.770000
7,5.78,9.630000,4.820000,5.6300,7.990000,1.410,3.730833,7.270,6.730000,6.560583
8,3.17,1.000000,5.990000,8.7200,9.810000,2.000,4.970000,3.990,8.750000,7.020000
9,7.26,9.810000,3.760000,8.9900,4.690000,1.250,4.840000,2.490,6.777500,6.050000


In [39]:
ranks_for_map_prediction = []
check = 0
count = 0
for i in predicted_ranks_for_map.index:
    #print(i)
    for c in predicted_ranks_for_map.columns:
        if(predicted_ranks_for_map.loc[i][c] == 1):
            ranks_for_map_prediction.append(c) 
            check = check+1
            count = count+1
            print(i)
            #break
            #print(i,c)
    #if(check>1):
        #print(i, predicted_ranks_for_map.loc[i])
        
        
    check = 0
print(count)
#print(len(ranks_for_map_prediction))
#print(len(predicted_ranks_for_map))

8
12
16
17
20
21
22
23
27
29
30
34
36
37
38
43
46
62
67
68
71
80
84
95
96
101
102
103
111
121
125
128
129
134
140
144
151
152
155
156
158
161
171
175
176
177
178
181
187
193
199
200
203
205
206
207
208
214
217
218
223
224
229
233
236
237
240
244
245
247
248
249
251
252
256
261
262
271
275
277
278
279
280
285
289
292
295
297
299
300
301
302
305
306
314
319
320
321
323
331
332
334
337
340
341
342
347
348
353
355
356
375
378
379
381
383
389
392
400
402
410
412
413
416
417
420
421
422
425
429
430
431
434
438
444
447
449
450
455
458
462
141
